In [1]:
import tensorflow as tf

In [2]:
# define the image size and batch size
img_size = (40, 40)
batch_size = 32

# Path to your dataset folder
data_dir = "C:/Users/ASUS/Desktop/Mixed_Marks"

In [3]:
# use image_dataset_from_directory to load the data and split it into training and testing sets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=img_size,
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=img_size,
    batch_size=batch_size)

Found 18135 files belonging to 8 classes.
Using 14508 files for training.
Found 18135 files belonging to 8 classes.
Using 3627 files for validation.


In [4]:
# inspect the class names and number of classes
class_names = train_ds.class_names
num_classes = len(class_names)
print('Class names:', class_names)
print('Number of classes:', num_classes)

Class names: ['0', '1', '2', '3', '4', '5', '6', '7']
Number of classes: 8


In [5]:
# convert images to grayscale
def grayscale(image, label):
    image = tf.image.rgb_to_grayscale(image)
    return image, label

train_ds = train_ds.map(grayscale)
val_ds = val_ds.map(grayscale)

# normalize pixel values to be between 0 and 1
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

In [6]:
# create the neural network
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(40, 40, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [7]:
# compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
# train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


454/454 [==============================] - 10s 20ms/step - loss: 1.0980 - accuracy: 0.6357 - val_loss: 0.5131 - val_accuracy: 0.8481
Epoch 2/10
454/454 [==============================] - 8s 18ms/step - loss: 0.3828 - accuracy: 0.8896 - val_loss: 0.3059 - val_accuracy: 0.9118
Epoch 3/10
454/454 [==============================] - 8s 18ms/step - loss: 0.2503 - accuracy: 0.9273 - val_loss: 0.2432 - val_accuracy: 0.9272
Epoch 4/10
454/454 [==============================] - 8s 18ms/step - loss: 0.1861 - accuracy: 0.9473 - val_loss: 0.1962 - val_accuracy: 0.9440
Epoch 5/10
454/454 [==============================] - 8s 19ms/step - loss: 0.1441 - accuracy: 0.9585 - val_loss: 0.1756 - val_accuracy: 0.9479
Epoch 6/10
454/454 [==============================] - 8s 18ms/step - loss: 0.1124 - accuracy: 0.9677 - val_loss: 0.1564 - val_accuracy: 0.9548
Epoch 7/10
454/454 [==============================] - 9s 19ms/step - loss: 0.0944 - accuracy: 0.9737 - val_loss: 0.1505 - val_accuracy: 0.9553
Epoch 8/1

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 38, 38, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 19, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 64)                1

In [10]:
model.save('MP.h5')